In [308]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from numpy import nan as NA

"""
from numpy import nan as NA
from numpy.random import randn
from scipy.stats import norm
from scipy import stats
from scipy.stats import chi2_contingency
from scipy.stats import f_oneway
"""


'\nfrom numpy import nan as NA\nfrom numpy.random import randn\nfrom scipy.stats import norm\nfrom scipy import stats\nfrom scipy.stats import chi2_contingency\nfrom scipy.stats import f_oneway\n'

In [309]:
#Carga de los datos

df_all_13 = pd.read_csv("../Tarea-1-Datos/data/raw/w13/metrocuadrado_all_w13.csv", engine='python')
df_furnished_13 = pd.read_csv("../Tarea-1-Datos/data/raw/w13/metrocuadrado_all_w13.csv", engine='python')
df_all_14 = pd.read_csv("../Tarea-1-Datos/data/raw/w14/metrocuadrado_all_w14.csv", engine='python')
df_furnished_14 = pd.read_csv("../Tarea-1-Datos/data/raw/w14/metrocuadrado_furnished_w14.csv", engine='python')
df_all_15 = pd.read_csv("../Tarea-1-Datos/data/raw/w15/metrocuadrado_all_w15.csv", engine='python')
df_furnished_15 = pd.read_csv("../Tarea-1-Datos/data/raw/w15/metrocuadrado_furnished_w15.csv", engine='python')
df_all_16 = pd.read_csv("../Tarea-1-Datos/data/raw/w16/metrocuadrado_all_w16.csv", engine='python')
df_furnished_16 = pd.read_csv("../Tarea-1-Datos/data/raw/w16/metrocuadrado_furnished_w16.csv", engine='python')
df_all_17 = pd.read_csv("../Tarea-1-Datos/data/raw/w17/metrocuadrado_all_w17.csv", engine='python')
df_furnished_17 = pd.read_csv("../Tarea-1-Datos/data/raw/w17/metrocuadrado_furnished_w17.csv", engine='python')

#Agregamos variable categórica que diga si viene o no de un archivo "furnished"

df_all_13['furnished']=0
df_all_14['furnished']=0
df_all_15['furnished']=0
df_all_16['furnished']=0
df_all_17['furnished']=0
df_furnished_13['furnished']=1
df_furnished_14['furnished']=1
df_furnished_15['furnished']=1
df_furnished_16['furnished']=1
df_furnished_17['furnished']=1


#generamos un sólo DataFrame

frames = [df_all_13, df_furnished_13, df_all_14, df_furnished_14, df_all_15, df_furnished_15, df_all_16, df_furnished_16, df_all_17, df_furnished_17]

df = pd.concat(frames)
df1 = df
#Eliminamos filas duplicadas

df = df.drop_duplicates(['property_type|rent_type|location', 'price', 'n_rooms', 'n_bath',
       'surface', 'details', 'url', 'metrocuadrado_index'])

#Reindexamos y rdenamos para mejorar la eficiencia

df = df.reset_index(drop=True)
df.sort_index(inplace=True)

#¿cuantas filas furnished no estaban repetidas?

len(df[df["furnished"]>0])


1947

In [310]:
#pasamos el precio de str a float
df["price"] = df["price"].replace('[\$,]', '', regex=True)
df["price"] = df["price"].replace('[\.,]', '', regex=True).astype(float)

#mismo ejercicio con la superficie y resto de columnas
df["surface"] = df["surface"].replace('[\ m2,]', '', regex=True).astype(float)

#si tiene más de 5 piezas o baños los agruparemos como que tienen 6
df["n_rooms"] = df["n_rooms"].replace('[\5+,]', '6', regex=True).astype(float)

df["n_bath"] = df["n_bath"].replace('[\5+,]', '6', regex=True).astype(float)


#df[df["n_rooms"].isnull()]["n_rooms"]

#Separamos la columna original en tres y la eliminamos luego

df["property_type"] = df['property_type|rent_type|location'].str.split(expand=True)[0]
df["property_type"] = df["property_type"].replace('[\,,]', '', regex=True)

df["rent_type"] = df['property_type|rent_type|location'].str.split(expand=True)[2]
df["rent_type"] = df["rent_type"].replace('[\,,]', '', regex=True)

dflocation = df['property_type|rent_type|location'].str.split(n=3).str[3]
df["location"] = dflocation=dflocation.iloc[:,].apply(lambda x: ''.join(x))

del df['property_type|rent_type|location']

df.head()

,price,n_rooms,n_bath,surface,details,url,metrocuadrado_index,furnished,property_type,rent_type,location
0,1050000.0,3.0,2.0,63.0,Excelente casa cerca de la calle 80 y Avenida ...,https://www.metrocuadrado.com/inmueble/arriend...,25.6,0,Casa,Arriendo,LA SOLEDAD NORTE BogotÃ¡ D.C..
1,3930000.0,3.0,2.0,100.0,Se arrienda casa bonita totalmente amoblada co...,https://www.metrocuadrado.com/inmueble/arriend...,57.3,0,Casa,Arriendo,CIUDAD SALITRE SUR-ORIENTAL BogotÃ¡ D.C..
2,750000.0,3.0,1.0,90.0,CASA DE TRES NIVELES TRES ALCOBAS SALA COMEDOR...,https://www.metrocuadrado.com/inmueble/arriend...,12.2,0,Casa,Arriendo,VILLA DE LOS ALPES II SECTOR BogotÃ¡ D.C..
3,3400000.0,5.0,4.0,71.0,casa ezquinera excelente ubicacion cinco alcob...,https://www.metrocuadrado.com/inmueble/arriend...,56.0,0,Casa,Arriendo,PARIS GAITAN BogotÃ¡ D.C..
4,1560000.0,4.0,3.0,96.0,"Excelente casa de tres niveles, buena ubicaciÃ...",https://www.metrocuadrado.com/inmueble/arriend...,47.1,0,Casa,Arriendo,BALCONES DE ORIENTE BogotÃ¡ D.C..


In [312]:


def price_per_m2 (row):
    if row['surface'] == 0 :
        return NA
    else:
        return row["price"]/row["surface"]

df['price per m2'] = df.apply (lambda row: price_per_m2 (row), axis=1)



,price,n_rooms,n_bath,surface,details,url,metrocuadrado_index,furnished,property_type,rent_type,location,price per m2
0,1050000.0,3.0,2.0,63.0,Excelente casa cerca de la calle 80 y Avenida ...,https://www.metrocuadrado.com/inmueble/arriend...,25.6,0,Casa,Arriendo,LA SOLEDAD NORTE BogotÃ¡ D.C..,16666.666667
1,3930000.0,3.0,2.0,100.0,Se arrienda casa bonita totalmente amoblada co...,https://www.metrocuadrado.com/inmueble/arriend...,57.3,0,Casa,Arriendo,CIUDAD SALITRE SUR-ORIENTAL BogotÃ¡ D.C..,39300.000000
2,750000.0,3.0,1.0,90.0,CASA DE TRES NIVELES TRES ALCOBAS SALA COMEDOR...,https://www.metrocuadrado.com/inmueble/arriend...,12.2,0,Casa,Arriendo,VILLA DE LOS ALPES II SECTOR BogotÃ¡ D.C..,8333.333333
3,3400000.0,5.0,4.0,71.0,casa ezquinera excelente ubicacion cinco alcob...,https://www.metrocuadrado.com/inmueble/arriend...,56.0,0,Casa,Arriendo,PARIS GAITAN BogotÃ¡ D.C..,47887.323944
4,1560000.0,4.0,3.0,96.0,"Excelente casa de tres niveles, buena ubicaciÃ...",https://www.metrocuadrado.com/inmueble/arriend...,47.1,0,Casa,Arriendo,BALCONES DE ORIENTE BogotÃ¡ D.C..,16250.000000


In [303]:
#pasamos el precio de str a float
df1["price"] = df1["price"].replace('[\$,]', '', regex=True)
df1["price"] = df1["price"].replace('[\.,]', '', regex=True).astype(float)

#mismo ejercicio con la superficie y resto de columnas
df1["surface"] = df1["surface"].replace('[\ m2,]', '', regex=True).astype(float)

#si tiene más de 5 piezas o baños los agruparemos como que tienen 6
df1["n_rooms"] = df1["n_rooms"].replace('[\5+,]', '6', regex=True).astype(float)

df1["n_bath"] = df1["n_bath"].replace('[\5+,]', '6', regex=True).astype(float)

sum(df1["surface"]==0)

1272

(23780, 8) (23780, 8)


In [91]:
df["furnished"].head

<bound method NDFrame.head of 0        0
0        1
0        0
0        0
0        0
        ..
12002    0
12003    0
12004    0
12005    0
12006    0
Name: furnished, Length: 18242, dtype: int64>

In [184]:
type(df["n_rooms"].loc[1])

str

In [ ]:
sum(df["n_bath"]==0)
def price_per_m2 (row):
    if row['surface'] == 0 :
        return NA
    else:
        return row["price"]/row["surface"]

df['price per m2'] = df.apply (lambda row: price_per_m2 (row), axis=1)